# Cars Tracking.

## Implementation steps:
1. Initial configuration of tools
  - choose tools for detection and tracking 
  - tune Background Subtractor
2. Video preprocessing
  - try different methods of preprocessing (grayscaling, blurring, highlighting)
  - tune blurring
3. Object Detection
4. Object Tracking 
3. Generation of a new video

## Used technical stack
- **For object detection**: K-nearest neighbours Background Subtractor from cv2 library 

With parameters `history=100` and `dist2Threshold=50` it showed better results than Gaussian Mixture-based Subtractor (BackgroundSubtractorMOG2) - the bounding boxes were more stable and precise.

- **For object tracking**: custom EuclideanDistTracker from [source](https://pysource.com/2021/01/28/object-tracking-with-opencv-and-python/). 

The tracker assigns a number (or ID) to each object tracked and draws the number near the object.

## MVP specifications:
1. Bounding boxes are stabilized with tuning of the Background Subtractor and Gaussian Blurring. 
2. The tracking algorithm is not perfect (there are a couple of miscalculations with assigning IDs).


In [ ]:
from google.colab import files
import cv2

In [ ]:
# source of the tracker.py code:
# https://pysource.com/2021/01/28/object-tracking-with-opencv-and-python/
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    #print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
uploaded = files.upload()

Saving video.mp4 to video.mp4


In [ ]:
# Upload a video
filename = list(uploaded.keys())[0]
cap = cv2.VideoCapture(filename)

if not cap.isOpened():
    print("Error opening video stream or file")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configure output video
output_filename = 'video_tracking.mp4'
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_size = (frame_width, frame_height)  # (684, 360)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_filename, fourcc, fps, frame_size)

# Configure the object detector and the tracker
object_detector = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=50, detectShadows=True)
tracker = EuclideanDistTracker()

while cap.isOpened():
    success, frame = cap.read()
    if success:

        # 1. Preprocess the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur_frame = cv2.GaussianBlur(gray_frame, ksize=(3, 3), sigmaX=cv2.BORDER_DEFAULT)

        # 2. Extract the region of interest (the road)
        x1, x2 = 0, frame_width
        y1, y2 = 130, 240
        roi = blur_frame[y1:y2, x1:x2]

        # 3. Object detection and contouring
        mask = object_detector.apply(roi)
        detections = []

        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            # Calculate area and remove small elements
            area = cv2.contourArea(cnt)
            if area > 80:
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h])

        # 4. Object tracking
        boxes_ids = tracker.update(detections)
        for box_id in boxes_ids:
            x, y, w, h, id = box_id
            cv2.putText(roi, str(id), (x-30, y+15), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)

            y += 130
            cv2.putText(frame, str(id), (x-30, y+15), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)

        output.write(frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
output.release()
print("Tracking is done!")
files.download(output_filename)

Tracking is done!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>